# Practice Model re training

In [1]:
import itertools
import os

import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

print(f"TF version: {tf.__version__}")
print(f"Hub version: {hub.__version__}")
print("GPU is", "available" if tf.test.is_gpu_available() else "Not available")

TF version: 2.9.1
Hub version: 0.12.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is Not available


### Model select (already training)

In [2]:
module_selection = ("mobilenet_v2_100_224", 224)
handle_base, pixels = module_selection
MODULE_HANDLE = f"https://tfhub.dev/google/imagenet/{handle_base}/feature_vector/4"
IMAGE_SIZE = (pixels, pixels)
print(f"Using {MODULE_HANDLE} with input size {IMAGE_SIZE}")

BATCH_SIZE = 32


Using https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4 with input size (224, 224)


### Set up flower dataset

In [4]:
data_dir = tf.keras.utils.get_file(
    'flower_phtos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True)

228813984/228813984 [==============================] - 100s 0us/step
